In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date, game_season
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from psycopg2.extras import execute_batch
from sql_queries import summary_table_insert
import urllib.parse
from datetime import datetime

In [75]:
def get_table_columns(table_element):
    print(table_element.get_attribute('innerHTML'))
    header_container = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="table-header-container", unique_element=True)
#     print(header_container.get_attribute('innerHTML'))
    header_elements = wait_for_element(source=header_container, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in header_elements]
    
    return header_names

def get_table_rows(table_element):
    table_rows = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    
    return rows

In [76]:
class Scraper:
    def __init__(self):
        self.driver = None
        self.conn = None
        self.cur = None
        self.table_data = None
        self.table_headers = None
        self.table_insert = ""
    
    def open_driver(self):
        self.driver = open_driver()
    
    def close_driver(self):
        if self.driver:
            close_driver(self.driver)
            
    def db_connect(self):
        self.conn, self.cur = db_connect()
    
    def close_db_connect(self):
        if self.conn and self.cur:
            close_db_connect(self.conn, self.cur)
            
    def extract(self, url):
        """sends requests to the url and scrapes the table elements (header and rows)"""
        self.driver.get(url)
        
        # scrape the page's main table
        root_element = wait_for_element(source=self.driver, search_by=By.ID, target="root", unique_element=True)
        data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)

        header_elements = get_table_columns(data_table)
        row_elements = get_table_rows(data_table)
        
        return header_elements, row_elements
    

    def load(self, data_rows):
        execute_batch(self.cur, self.table_insert, data_rows)

In [77]:
class SummaryScraper(Scraper):
    def __init__(self):
        super().__init__()
        self.table_insert = summary_table_insert
        self.base_url = "https://www.nhl.com/stats/teams?"
        self.url_dict = {
            "aggregate": 0,
            "reportType": "game",
            "page": 0,
            "pageSize": 100
        }
    
    def build_url(self, start, end=None, playoffs=False):
        if playoffs:
            self.url_dict["gameType"] = 3
        else:
            self.url_dict["gameType"] = 2
            
        if isinstance(start, int):
            self.url_dict["seasonFrom"] = start
            
            if isinstance(end, int):
                self.url_dict["seasonTo"] = end
            else:
                self.url_dict["seasonTo"] = f"{datetime.now().year}{datetime.now().year+1}"
            
            self.url_dict["dateFromSeason"] = []
        
        elif isinstance(start, str) and start.isnumeric():
            self.url_dict["seasonFrom"] = start
            
            if isinstance(end, str) and end.isnumeric():
                self.url_dict["seasonTo"] = end
            else:
                self.url_dict["seasonTo"] = start
            
            self.url_dict["dateFromSeason"] = []
            
        elif isinstance(start, datetime):            
            self.url_dict["dateFrom"] = start.strftime("%Y-%m-%d")
            
            if isinstance(end, datetime):
                self.url_dict["dateTo"] = end.strftime("%Y-%m-%d")
            else:
                self.url_dict["dateTo"] = datetime.now().date()
        
        url = self.base_url + urllib.parse.urlencode(self.url_dict)
        
        return url
        
    def transform(self, headers, row_elements):
        table_data = []
        
        for row in row_elements:
            row_cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
            row_values = [cell.text for cell in row_cells]
            
            values_map = list(zip(headers, row_values))
            map_dict = parse_record(values_map)
            
            self.cur.execute("SELECT abbreviation FROM teams WHERE team = %s;", (map_dict["Team"],))
            team_abbreviation = self.cur.fetchone()[0]
            
            ordered_data_list = [
                team_abbreviation,
                map_dict["season"],
                map_dict["game_date"],
                map_dict["home_game"],
                map_dict["opponent"],
                map_dict["W"],
                map_dict["L"],
                map_dict["T"],
                map_dict["OT"],
                map_dict["P"],
                map_dict["P%"],
                map_dict["RW"],
                map_dict["ROW"],
                map_dict["S/O Win"],
                map_dict["GF"],
                map_dict["GA"],
                map_dict["PP%"],
                map_dict["PK%"],
                map_dict["Net PP%"],
                map_dict["Net PK%"],
                map_dict["Shots/GP"],
                map_dict["GA/GP"],
                map_dict["FOW%"]
            ]
        
            table_data.append(ordered_data_list)
        
        return table_data
    
    def etl(self, start, playoffs=False):
        url = self.build_url(start=start)
        
        # the chrome driver and database cursor are used in multiple scripts
        self.open_driver()
        self.db_connect()
        
        table_headers, table_rows = self.extract(url)

        parsed_data = self.transform(table_headers, table_rows)        
        
        # execute batch loading
        self.load(parsed_data)
        
        # close the driver and the cursor
        self.close_driver()
        self.close_db_connect()
        
        return parsed_data

In [78]:
scraper = SummaryScraper()

In [79]:
summary_table_data = scraper.etl(2021)

<div class="table-header-container"><div class="rt-thead -header" style="min-width: 1311px;"><div class="rt-tr" role="row"><div colspan="1" role="columnheader" class="rt-th index-column th-fixed" style="flex: 46 0 auto; width: 46px; min-width: 46px; max-width: 60px; left: 0px; cursor: initial;"><div content="Hold shift while selecting a column to sort by multiple columns." class="styles__RTHeaderCell-sc-inbkew-0 cCLsoX rt-header-cell"><div class="styles__InfoIcon-sc-inbkew-1 OpBOI">i</div></div></div><div colspan="1" role="columnheader" title="Toggle SortBy" class="rt-th th-fixed th-fixed-last" style="flex: 155 0 auto; width: 155px; min-width: 155px; max-width: 200px; left: 46px; cursor: pointer;"><div title="Team" class="rt-header-cell rt-header-cell__left">Team<br></div></div><div colspan="1" role="columnheader" title="Toggle SortBy" class="rt-th" style="flex: 142 0 auto; width: 142px; min-width: 142px; max-width: 160px; cursor: pointer;"><div title="Game" class="rt-header-cell rt-he

NameError: name 'month' is not defined

In [ ]:
len(summary_table_data)